In [12]:
import keras
import h5py
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import backend as K

batch_size = 128
epochs = 20
infodir = 'C:/Science Research/xraynet_data/images/images'

train_data_dir = f'{infodir}/train'
test_data_dir =  f'{infodir}/test'

train_h5_path = f'{infodir}/train.h5'
test_h5_path = f'{infodir}/test.h5'
MODEL_NAME = 'xrayNet-keras-v1.5' #gve the model a name
# input image dimensions
img_size = 128
input_shape = (img_size,img_size, 1)

In [2]:
import h5py
h5f = h5py.File(train_h5_path, 'r')
X = h5f['X']
Y = h5f['Y']

h5ft = h5py.File(test_h5_path, 'r')
test_X = h5ft['X']
test_Y = h5ft['Y']

In [3]:
Y = [l[0] for l in Y]
test_Y = [l[0] for l in test_Y]

In [4]:
from tflearn.data_utils import shuffle

X,Y = shuffle(X,Y)
test_X,test_Y = shuffle(test_X,test_Y)

X = X.reshape([-1,img_size,img_size,1])
test_X = test_X.reshape([-1,img_size,img_size,1])

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [8]:
tbCallBack = keras.callbacks.TensorBoard(log_dir=f'./TBLOGS/{MODEL_NAME}', histogram_freq=0,  
          write_graph=True, write_images=True)

In [6]:
from keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(128))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Activation('relu'))


model.add(Dense(1))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

In [13]:
model.fit(X, Y,
          epochs=epochs,
          batch_size=batch_size,
          verbose=1,
          shuffle = True, 
          validation_data=(test_X,test_Y),    #data to be be used as vailidation
          callbacks=[tbCallBack]) #pushes data to tensorboard

Train on 100000 samples, validate on 3416 samples
Epoch 1/40
100000/100000 [==============================] - 269s 3ms/step - loss: 0.2151 - acc: 0.9077 - val_loss: 0.8776 - val_acc: 0.5665
Epoch 2/40
100000/100000 [==============================] - 269s 3ms/step - loss: 0.2002 - acc: 0.9147 - val_loss: 0.9990 - val_acc: 0.5946
Epoch 3/40
100000/100000 [==============================] - 269s 3ms/step - loss: 0.1891 - acc: 0.9201 - val_loss: 0.8309 - val_acc: 0.5992
Epoch 4/40
100000/100000 [==============================] - 269s 3ms/step - loss: 0.1761 - acc: 0.9260 - val_loss: 0.9596 - val_acc: 0.6036
Epoch 5/40
100000/100000 [==============================] - 269s 3ms/step - loss: 0.1650 - acc: 0.9294 - val_loss: 0.8545 - val_acc: 0.6077
Epoch 6/40
100000/100000 [==============================] - 269s 3ms/step - loss: 0.1601 - acc: 0.9324 - val_loss: 1.1779 - val_acc: 0.5975
Epoch 7/40
100000/100000 [==============================] - 269s 3ms/step - loss: 0.1537 - acc: 0.9344 - val_l

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-b315d47de08a>", line 7, in <module>
    callbacks=[tbCallBack]) #pushes data to tensorboard
  File "D:\anaconda\lib\site-packages\keras\models.py", line 960, in fit
    validation_steps=validation_steps)
  File "D:\anaconda\lib\site-packages\keras\engine\training.py", line 1657, in fit
    validation_steps=validation_steps)
  File "D:\anaconda\lib\site-packages\keras\engine\training.py", line 1213, in _fit_loop
    outs = f(ins_batch)
  File "D:\anaconda\lib\site-packages\keras\backend\tensorflow_backend.py", line 2357, in __call__
    **self.session_kwargs)
  File "D:\anaconda\lib\site-packages\tensorflow\python\client\session.py", line 889, in run
    run_metadata_ptr)
  File "D:\anaconda\lib\site-packages\tensorflow\python\client\session.py", line 1120, in _run
    feed_dict_

KeyboardInterrupt: 

In [ ]:
model.save('{}.{}.model'.format(MODEL_NAME,'')) #save the model with the name in the working directory

In [48]:
#print(model.predict(np.array([X[-1]]))[0])
#print(Y[-1])

pred_Y = model.predict(test_X)
pred_Y = (pred_Y > 0.5)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_Y, pred_Y)

In [49]:
print(cm)

[[ 815  893]
 [ 437 1271]]


### 0 == Normal

### 1 == Abnormal

### Abnormalness